In [1]:
import pandas as pd
import requests
import json
import csv
import time
import datetime
from datetime import date

In [2]:
# date
after_start_date = date.fromisoformat('2016-04-01')
before_end_date = date.fromisoformat('2019-04-30')
after_start_seconds = (after_start_date -  date.fromisoformat('1970-01-01')).total_seconds()
before_end_seconds = (before_end_date -  date.fromisoformat('1970-01-01')).total_seconds()

print('after_start_seconds: ', after_start_seconds)
print('before_end_seconds: ', before_end_seconds)

after_start_seconds:  1459468800.0
before_end_seconds:  1556582400.0


# Retrieve the submissions

In [3]:
def getPushshiftData(query, after, before, sub):
    url = 'https://api.pushshift.io/reddit/search/submission/?title='+\
          str(query)+'&size=1000&after='+str(after)+'&before='+str(before)+'&subreddit='+str(sub)+'&sort=asc'
#     url = 'https://api.pushshift.io/reddit/search/submission/?title='+\
#           str(query)+'&size=1000'+'&subreddit='+str(sub)
#     print(url)
    r = requests.get(url)
#     print(r)
    data = json.loads(r.text)
    return data['data']

def getPushshiftData_noSubreddit(query, after, before):
    url = 'https://api.pushshift.io/reddit/search/submission/?title='+\
          str(query)+'&size=1000&after='+str(after)+'&before='+str(before)+'&sort=asc'
#     url = 'https://api.pushshift.io/reddit/search/submission/?title='+\
#           str(query)+'&size=1000'+'&subreddit='+str(sub)
#     print(url)
    r = requests.get(url)
    print(r)
    data = json.loads(r.text)
    return data['data']

In [4]:
def collectSubData(subm):
    subData = list() #list to store data points
    title = subm['title']
    url = subm['url']
    try:
        flair = subm['link_flair_text']
    except KeyError:
        flair = "NaN"    
    author = subm['author']
#     author_fullName = subm['author_fullname']
    sub_id = subm['id']
    score = subm['score']
    try:
        subreddit = subm['subreddit']
    except KeyError:
        subreddit = "None"
        
    try:
        self_text = subm['selftext']
    except KeyError:
        self_text = "None"
        
    created = datetime.datetime.fromtimestamp(subm['created_utc']) #1520561700.0
    numComms = subm['num_comments']
    permalink = subm['permalink']
    
    subData.append((sub_id, title, self_text, url, author, score,subreddit, created,numComms,permalink,flair))
    subStats[sub_id] = subData

In [5]:
# keywords list
# keywords = ['vape', 'e liquid', 'e juice', 'ejuice', 'electronic cigarette', 'eliquid', 'e cig', 'ecig', 'e hookah', 'electronic cig', 'ehookah']
# keywords = ['New Jersey']

# Category keyword
synonym_ecig = ['electronic-cigarette', 'electronic cigarette', 'electronic cig', ' e-cig', ' ecig', ' e cig', 'e-cigarette', 'ecigarette', ' e cigarette', 'e-juice', ' ejuice', ' e juice', 'e-liquid', ' eliquid', ' e liquid', ' e-smoke', ' esmoke', ' e smoke', ' vape', 'vaper', 'vaping', 'vape-juice',
                'vape-liquid', ' vapor', 'vaporizer', 'boxmod', 'cloud chaser', 'cloudchaser', 'smoke assist', 'ehookah', 'e-hookah', ' e hookah', 'stillblowingsmoke', 'still blowing smoke', 'cherry tip cigarillo', '#e-cig', '#ecig', '#e cigar', '#ejuice', '#e juice', '#eliquid', '#e liquid', '#vapor', '#ehookah', '#e hookah', 'smoke pod']
# remove ' fin '
brand_ecig = ['juul', 'vaporfi', 'vype pebble', 'v2 cig', 'v2cigs', 'v2 cigs', 'halocigs', ' njoy', 'markten', 'vuse', ' tryst', 'atomizer', 'cartomizer', 'south beach smoke', 'eversmoke', 'joye510', 'joye 510', 'joyetech', 'logicecig', 'smartsmoker', ' mistic', 'smokestiks', '21st century smoke', 'logic black label',
              ' finiti', 'nicotek', 'cigirex', 'ciga&blu', 'cig&blu', 'logic&cig', 'e-swisher', 'e swisher', 'eswisher', 'ezsmoker', 'ez&cig', 'green smoke', 'cigalectric', 'xhale o2', 'xhaleo2', 'cig2o', 'green smart living', 'greensmartliving', 'swisher blk', 'grimmgreen', '#njoy', '#fin ', '#finiti']
policy_ecig = ['sb140', 'sb 140', 'sb24', 'sb 24']
cessation_ecig = ['notblowingsmoke', 'not blowing smoke',
                  'tobaccofreekids', 'notareplacement']

def remove_space_and_conjunction(ecig_keywords):
    temp_keywords_list = []
    for keyword in ecig_keywords:
        keyword = keyword.strip()
        keyword = keyword.replace('&', ' ')
        keyword = keyword.replace('-', ' ')
        keyword = keyword.replace('#', '')
#         if ('&' in keyword) or ('-' in keyword) or ('#' in keyword):
#             continue
        temp_keywords_list.append(keyword)
    temp_keywords_list = list(set(temp_keywords_list)) # remove duplication
    return temp_keywords_list

# ecig keywords in total
ecig_keywords = []
ecig_keywords.extend(synonym_ecig)
ecig_keywords.extend(brand_ecig)
ecig_keywords.extend(policy_ecig)
ecig_keywords.extend(cessation_ecig)
ecig_keywords = remove_space_and_conjunction(ecig_keywords)

### Search keyword without specific subreddit

In [6]:
# # query = "electronic-cigarette"
# # sub = "electronic_cigarette"

# for query in keywords:
#     print('--------------------------------------')
#     print('Query: ', query)
#     after = str(after_start_seconds).split('.')[0]
#     before = str(before_end_seconds).split('.')[0]

#     subStats = {}
#     subreddit_dict = {}

#     data = getPushshiftData_noSubreddit(query, after, before)
#     # Will run until all posts have been gathered 
#     # from the 'after' date up until before date
#     while len(data) > 0:
#         for submission in data:
# #             print(submission)
# #             print(submission['title'])
# #             print(submission['subreddit'])
# #             print('-----------')
#             collectSubData(submission)
#     #         print(submission)
#             try:
#                 subreddit = submission['subreddit']
#             except KeyError:
#                 subreddit = "None"
#             subreddit_dict[subreddit] = subreddit_dict.get(subreddit, 0) + 1
#     #         break

#         # Prepare for next iteration
#         # print(str(datetime.datetime.fromtimestamp(data[-1]['created_utc'])))
#         after = data[-1]['created_utc']
#         data = getPushshiftData_noSubreddit(query, after, before)

#     print(len(data))
#     print('length of subStats.keys(): ', len(subStats.keys()))
# #     print('subreddit_dict: ', subreddit_dict)
#     print('Sorted: ', sorted(subreddit_dict.items(), key=lambda d: d[1], reverse=True))
#     break
# #     print('Sorted: ', sorted(subreddit_dict.items(), key=lambda d: d[1], reverse=True))


### Specific subreddit

In [11]:
sub = "sanfrancisco"
subCount = 0
subStats = {}

for query in ecig_keywords:
    print('--------------------------------------')
    print('Query: ', query)
    after = str(after_start_seconds).split('.')[0]
    before = str(before_end_seconds).split('.')[0]

    data = getPushshiftData(query, after, before, sub)
    # Will run until all posts have been gathered 
    # from the 'after' date up until before date
    while len(data) > 0:
        for submission in data:      
            collectSubData(submission)
    #         print(submission)
    #         subCount+=1
    #         break
        # Calls getPushshiftData() with the created date of the last submission
        print(len(data))
        print(str(datetime.datetime.fromtimestamp(data[-1]['created_utc'])))
        after = data[-1]['created_utc']
        data = getPushshiftData(query, after, before, sub)
    #     break

    print(len(data))
    time.sleep(.400)
    

--------------------------------------
Query:  halocigs
0
--------------------------------------
Query:  logicecig
0
--------------------------------------
Query:  south beach smoke
0
--------------------------------------
Query:  sb 24
0
--------------------------------------
Query:  vape liquid
0
--------------------------------------
Query:  21st century smoke
0
--------------------------------------
Query:  notblowingsmoke
0
--------------------------------------
Query:  joye 510
0
--------------------------------------
Query:  logic cig
0
--------------------------------------
Query:  joyetech
0
--------------------------------------
Query:  vape juice
0
--------------------------------------
Query:  e cig
0
--------------------------------------
Query:  e swisher
0
--------------------------------------
Query:  sb24
0
--------------------------------------
Query:  boxmod
0
--------------------------------------
Query:  atomizer
0
--------------------------------------
Query:  e c

In [12]:
print('length of subStats.keys(): ', len(subStats.keys()))

length of subStats.keys():  23


In [13]:
print(str(len(subStats)) + " submissions have added to list")
print("1st entry is:")
print(list(subStats.values())[0][0][1] + " created: " + str(list(subStats.values())[0][0][5]))
print("Last entry is:")
print(list(subStats.values())[-1][0][1] + " created: " + str(list(subStats.values())[-1][0][5]))

23 submissions have added to list
1st entry is:
SF e-cigarette maker Juul has 60 days to show FDA how it will reduce youth sales created: 31
Last entry is:
Microwaved Fish Was Once the Workplace Aggravation—Now It’s Vaping created: 1


In [14]:
def save_subStats(fileName, subStats):
    upload_count = 0
    location = "/mnt/volume-second-5T/subreddits/"
#     print("input filename of submission file, please add .csv")
#     filename = input()
    fileName = fileName + ".csv"
    file = location + fileName
    with open(file, 'w', newline='', encoding='utf-8') as file: 
        a = csv.writer(file, delimiter=',')
        headers = ["Post_ID","Title","Self_text", "Url","Author","Score","Subreddit","Publish_date","Number_of_comments","Permalink","Flair"]
        a.writerow(headers)
        for sub in subStats:
            a.writerow(subStats[sub][0])
            upload_count+=1
            
        print(str(upload_count) + " submissions have been uploaded")
        
save_subStats("sanfrancisco_ecig_keywords", subStats)

23 submissions have been uploaded
